## FABlib API References Examples

- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete) 

In [1]:
import datetime
import json
import asyncio

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

User: koufalex@gmail.com bastion key is valid!
Configuration is valid


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,koufalex_0000215529
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


In [2]:
%%time
slice_name = 'DYNAMOS-on-FABRIC'
image = "default_ubuntu_24"

# Please adhere to the following regex for naming: /[a-z][a-z0-9]+/
# note: see above, renamed the agent names to only have hyphens, not underscores 

node_configurations = [
    {
        "type": "control",
        "cores": 2,
        "ram": 8,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "dynamos",
        "cores": 4,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "agent",
        "name": "server",
        "cores": 4,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "agent",
        "name": "clientone",
        "cores": 4,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "agent",
        "name": "clienttwo",
        "cores": 8,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",
        "host": "losa-w3.fabric-testbed.net",
    },
    {
        "type": "agent",
        "name": "clientthree",
        "cores": 4,
        "ram": 16,
        "disk": 20,
        "site": "LOSA",  # TOKY seems to not be that responsive
        "host": "losa-w3.fabric-testbed.net",  # had to change this too 
    }
]

sites = list(set([configuration["site"] for configuration in node_configurations]))
agents = [configuration["name"] for configuration in node_configurations if configuration["type"] == "agent"]
thirdparties = [configuration["name"] for configuration in node_configurations if configuration["type"] == "thirdparty"]

def create_node(slice, configuration):
    if (configuration["type"] == "control"): 
        configuration["name"] = "control"

    if (configuration["type"] == "dynamos"): 
        configuration["name"] = "dynamos"
    
    return slice.add_node(name=configuration["name"], 
                          site=configuration["site"], 
                          host=configuration["host"], 
                          cores=configuration["cores"], 
                          ram=configuration["ram"], 
                          disk=configuration["disk"], 
                          validate=True, 
                          raise_exception=True, 
                          image=image)
    

CPU times: user 14 μs, sys: 0 ns, total: 14 μs
Wall time: 18.6 μs


In [3]:
%%time
# Create a slice
slice = fablib.new_slice(name=slice_name)

# Add Nodes with the specific variables
# Also validate the node can be created and raise an exception in case of failure
print('Adding nodes...')
nodes = [create_node(slice, configuration) for configuration in node_configurations]
nodes_per_site = [
    (site, [node for node in nodes if node.get_site() == site])
    for site in sites
]

print('Adding network interfaces...')
interfaces_per_site = [
    (site, [node.add_component(model='NIC_Basic', name='NIC').get_interfaces()[0] for node in nodes])
    for (site, nodes) in nodes_per_site
]

print('Adding network...')
networks = [
    slice.add_l3network(name=f'Network-{site}', interfaces=interfaces, type="IPv4")
    for (site, interfaces) in interfaces_per_site
]

print(networks, [n.get_gateway() for n in networks], [n.get_subnet() for n in networks])

# Calculate the lease end time for 2 weeks from now with timezone information
lease_end_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(weeks=2)

# Submit the slice, using an end date 2 weeks from now (the current maximum lease time) 
# to make sure that the slice can be used for a longer period of time. Progress shows an indicator of the current progression.
# Wait until the state is finished and use an interval (it may take some time before the slice and nodes are created)
print('Creating slice...')
slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text', lease_end_time=lease_end_time);


Retry: 8, Time: 247 sec


ID,495f5c4f-1988-429e-85eb-c1cdce7d660e
Name,DYNAMOS-on-FABRIC
Lease Expiration (UTC),2025-10-21 10:13:02 +0000
Lease Start (UTC),2025-10-07 10:13:02 +0000
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
State,StableOK
Email,koufalex@gmail.com
UserId,9e537cc1-dc5e-43e1-b4b2-eca83032df0c


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
9aac0a23-80a4-48a5-83b6-15473149eb96,clientone,4,16,100,default_ubuntu_24,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe15:1e28,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe15:1e28,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
06e8d178-f302-4705-933f-d4eb0d832f16,clientthree,4,16,100,default_ubuntu_24,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe2a:2190,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe2a:2190,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
80746aa7-1c1b-4341-b735-1bb24df63e82,clienttwo,8,16,100,default_ubuntu_24,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fea4:dbc3,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fea4:dbc3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
fee912f7-b48b-41ce-9c63-2645a6f6cf6f,control,2,8,100,default_ubuntu_24,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe19:b747,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe19:b747,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
40eb0b25-53b8-4f0b-892f-11a26dd87065,dynamos,4,16,100,default_ubuntu_24,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fef2:4910,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fef2:4910,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0e6fc382-69aa-4757-85c1-37feffcfa563,server,4,16,100,default_ubuntu_24,qcow2,losa-w3.fabric-testbed.net,LOSA,ubuntu,2001:400:a100:3070:f816:3eff:fe63:3953,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3070:f816:3eff:fe63:3953,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
39ba3edc-ea5b-490a-aca7-d02e1ac273e3,Network-LOSA,L3,FABNetv4,LOSA,10.137.8.0/24,10.137.8.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
control-NIC-p1,p1,control,Network-LOSA,100,config,,06:BA:43:20:27:43,enp6s0,enp6s0,fe80::4ba:43ff:fe20:2743,4,HundredGigE0/0/0/9
dynamos-NIC-p1,p1,dynamos,Network-LOSA,100,config,,06:FE:5D:78:95:3D,enp6s0,enp6s0,fe80::4fe:5dff:fe78:953d,4,HundredGigE0/0/0/9
server-NIC-p1,p1,server,Network-LOSA,100,config,,0A:03:50:C9:00:F2,enp6s0,enp6s0,fe80::803:50ff:fec9:f2,4,HundredGigE0/0/0/9
clientone-NIC-p1,p1,clientone,Network-LOSA,100,config,,12:A1:81:B6:42:22,enp6s0,enp6s0,fe80::10a1:81ff:feb6:4222,4,HundredGigE0/0/0/9
clienttwo-NIC-p1,p1,clienttwo,Network-LOSA,100,config,,16:56:4E:A6:40:D4,enp6s0,enp6s0,fe80::1456:4eff:fea6:40d4,4,HundredGigE0/0/0/9
clientthree-NIC-p1,p1,clientthree,Network-LOSA,100,config,,1A:02:51:44:43:0F,enp6s0,enp6s0,fe80::1802:51ff:fe44:430f,4,HundredGigE0/0/0/9



Time to print interfaces 258 seconds
CPU times: user 1min 12s, sys: 361 ms, total: 1min 13s
Wall time: 5min 5s


In [4]:
%%time
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();

nodes_and_network_per_site = [
    (site, [node for node in nodes if node.get_site() == site], slice.get_network(name=f"Network-{site}"))
    for site in sites
]
networks = [network for (_, _, network) in nodes_and_network_per_site]

nodes_network_ips_per_site = [
    (site, nodes, network, network.get_available_ips(len(nodes)))
    for (site, nodes, network) in nodes_and_network_per_site
]

CPU times: user 1.23 s, sys: 4 ms, total: 1.24 s
Wall time: 1.69 s


In [5]:
%%time
def assign_ip(site, network, available_ips, node):
    interface = node.get_interface(network_name=f"Network-{site}")
    address = available_ips.pop(0)
    network_gateway = network.get_gateway()
    network_subnet = network.get_subnet()

    network.allocate_ip(address)
    interface.ip_addr_add(addr=address, subnet=network_subnet)
    node.ip_route_add(subnet=network_subnet, gateway=network_gateway)

    # For the multisite IPv4 connection
    for network in networks:
        node.ip_route_add(subnet=network.get_subnet(), gateway=network_gateway)

    return address

ips = [assign_ip(site, network, ips, node) for (site, nodes, network, ips) in nodes_network_ips_per_site for node in nodes];

CPU times: user 2.07 s, sys: 74.2 ms, total: 2.15 s
Wall time: 25.2 s


In [6]:
%%time
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();

for node in nodes:
    ssh_command = node.get_ssh_command().replace(
        "-i /home/fabric/work/fabric_config/slice_key", "-i ~/.ssh/keys/FABRIC-slice_key"
    ).replace(
        "-F /home/fabric/work/fabric_config/ssh_config ", ""
    )
    
    print(ssh_command);

ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe19:b747
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fef2:4910
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe63:3953
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe15:1e28
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fea4:dbc3
ssh -i ~/.ssh/keys/FABRIC-slice_key ubuntu@2001:400:a100:3070:f816:3eff:fe2a:2190
CPU times: user 8.84 s, sys: 36.9 ms, total: 8.87 s
Wall time: 9.2 s


In [7]:
%%time
print("Uploading the node setup...")
threads = [node.upload_file_thread(local_file_path="node_scripts/node_setup.sh", remote_file_path="setup.sh")
           for node in nodes]
[thread.result() for thread in threads]


Uploading the node setup...
CPU times: user 130 ms, sys: 21.2 ms, total: 152 ms
Wall time: 2.07 s


[<SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1759832964 mtime=1759832964 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1759832964 mtime=1759832964 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1759832964 mtime=1759832964 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1759832964 mtime=1759832964 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1759832964 mtime=1759832965 ]>,
 <SFTPAttributes: [ size=1222 uid=1000 gid=1000 mode=0o100664 atime=1759832964 mtime=1759832965 ]>]

In [8]:
%%time
print("Executing the node setup...")
threads = [node.execute_thread(f"chmod +x setup.sh && ./setup.sh")
           for node in nodes]
[thread.result() for thread in threads]

Executing the node setup...
CPU times: user 3.98 s, sys: 811 ms, total: 4.79 s
Wall time: 3min 28s


[("Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease\nGet:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]\nGet:3 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]\nGet:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]\nGet:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 Packages [15.0 MB]\nGet:6 http://security.ubuntu.com/ubuntu noble-security/main amd64 Packages [1207 kB]\nGet:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe Translation-en [5982 kB]\nGet:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 Components [3871 kB]\nGet:9 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 c-n-f Metadata [301 kB]\nGet:10 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse amd64 Packages [269 kB]\nGet:11 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse Translation-en [118 kB]\nGet:12 http://nova.clouds.archive

In [9]:
%%time
def get_ip(node):
    interface = node.get_interface(network_name=f"Network-{node.get_site()}")
    return interface.get_ip_addr()

nodes_dict= dict()

for node in nodes[:]:
    ip = get_ip(node)
    name = node.get_name()
    nodes_dict[name] = {"ip": ip, "node": node}
    print(f"{name}: {ip}")

print(nodes_dict)


control: 10.137.8.2
dynamos: 10.137.8.3
server: 10.137.8.4
clientone: 10.137.8.5
clienttwo: 10.137.8.6
clientthree: 10.137.8.7
{'control': {'ip': '10.137.8.2', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d0be490fcd0>}, 'dynamos': {'ip': '10.137.8.3', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d0be4e70710>}, 'server': {'ip': '10.137.8.4', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d0be4e07ad0>}, 'clientone': {'ip': '10.137.8.5', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d0be4e984d0>}, 'clienttwo': {'ip': '10.137.8.6', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d0be4e7b090>}, 'clientthree': {'ip': '10.137.8.7', 'node': <fabrictestbed_extensions.fablib.node.Node object at 0x7d0c040807d0>}}
CPU times: user 1.57 s, sys: 26.1 ms, total: 1.59 s
Wall time: 11 s


In [10]:
%%time
nodes_dict["control"]["node"].upload_file(local_file_path="node_scripts/install_k9s.sh", remote_file_path="k9s.sh")
nodes_dict["control"]["node"].execute(f"chmod +x k9s.sh && ./k9s.sh");

--2025-10-07 10:34:47--  https://github.com/derailed/k9s/releases/download/v0.32.5/k9s_linux_amd64.deb
Resolving github.com (github.com)... 2600:2701:5000:5001::8c52:7003, 140.82.112.3
Connecting to github.com (github.com)|2600:2701:5000:5001::8c52:7003|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/167596393/7cc41638-6a22-4598-9b02-646efaaa1053?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-07T11%3A09%3A00Z&rscd=attachment%3B+filename%3Dk9s_linux_amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-07T10%3A08%3A36Z&ske=2025-10-07T11%3A09%3A00Z&sks=b&skv=2018-11-09&sig=Sluhq0yHWclCTWuaKOMM5Kz%2BeKDAe%2BBIuCpkjMoIXqM%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1OTgzNTA4NywibmJmIjoxNz

In [11]:
# the order of the nodes is not the same as in the configuration 
# ips
# names
# names_and_ips = {n:i for n,i in names_and_ips}
# print(names_and_ips)

In [11]:
%%time
inventory = (
    f"[kube_control_plane]\n"
    f"control ansible_host={nodes_dict['control']['ip']} ip={nodes_dict['control']['ip']} etcd_member_name=etcd1\n"
    f"\n"
    f"[etcd:children]\n"
    f"kube_control_plane\n"
    f"\n"
    f"[kube_node]\n"
    f"dynamos ansible_host={nodes_dict['dynamos']['ip']} ip={nodes_dict['dynamos']['ip']}\n"
)

for node_name in nodes_dict.keys():
    if node_name not in ["dynamos", "control"]:
        inventory += f"{node_name} ansible_host={nodes_dict[node_name]['ip']} ip={nodes_dict[node_name]['ip']}\n"

with open('kubespray/inventory.ini', 'w') as f:
    f.write(inventory)

CPU times: user 717 μs, sys: 2.01 ms, total: 2.73 ms
Wall time: 1.56 ms


In [12]:
print(inventory)

[kube_control_plane]
control ansible_host=10.137.8.2 ip=10.137.8.2 etcd_member_name=etcd1

[etcd:children]
kube_control_plane

[kube_node]
dynamos ansible_host=10.137.8.3 ip=10.137.8.3
server ansible_host=10.137.8.4 ip=10.137.8.4
clientone ansible_host=10.137.8.5 ip=10.137.8.5
clienttwo ansible_host=10.137.8.6 ip=10.137.8.6
clientthree ansible_host=10.137.8.7 ip=10.137.8.7



In [13]:
%%time
nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/control_kubespray_setup.sh", remote_file_path="kubespray_setup.sh");
nodes_dict['control']['node'].execute("chmod +x kubespray_setup.sh && ./kubespray_setup.sh");

nodes_dict['control']['node'].upload_file(local_file_path="kubespray/inventory.ini", remote_file_path="kubespray/inventory/dynamos/inventory.ini");
nodes_dict['control']['node'].upload_file(local_file_path="kubespray/ansible.cfg", remote_file_path="kubespray/ansible.cfg");
nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/dot_kube.sh", remote_file_path="dot_kube.sh");
nodes_dict['control']['node'].execute("chmod +x ./dot_kube.sh");
nodes_dict['control']['node'].upload_file(local_file_path="/home/fabric/work/fabric_config/slice_key", remote_file_path="/home/ubuntu/.ssh/slice_key");
nodes_dict['control']['node'].execute("chmod 600 /home/ubuntu/.ssh/slice_key");

Cloning into 'kubespray'...
branch 'release-2.27' set up to track 'origin/release-2.27'.
Switched to a new branch 'release-2.27'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 136.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.6/219.6 kB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 40.2 MB/s eta 0:00:00
ansible [core 2.16.14]
  config file = /home/ubuntu/kubespray/ansible.cfg
  configured module search path = ['/home/ubuntu/kubespray/library']
  ansible pyth

In [14]:
%%time
# 27 mins 
nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/start_kubespray.sh", remote_file_path="start.sh");
nodes_dict['control']['node'].execute(f"chmod +x start.sh && ./start.sh");

[WARNING]: While constructing a mapping from
/home/ubuntu/kubespray/roles/bootstrap-os/tasks/main.yml, line 29, column 7,
found a duplicate dict key (paths). Using last defined value only.
[WARNING]: Skipping callback plugin 'ara_default', unable to load
Using /home/ubuntu/kubespray/ansible.cfg as config file

PLAY [Check Ansible version] ***************************************************
Tuesday 07 October 2025  10:38:20 +0000 (0:00:00.008)       0:00:00.008 ******* 

TASK [Check 2.16.4 <= Ansible version < 2.17.0] ********************************
ok: [dynamos] => {
    "changed": false,
    "msg": "All assertions passed"
}
Tuesday 07 October 2025  10:38:20 +0000 (0:00:00.020)       0:00:00.029 ******* 

TASK [Check that python netaddr is installed] **********************************
ok: [dynamos] => {
    "changed": false,
    "msg": "All assertions passed"
}
Tuesday 07 October 2025  10:38:20 +0000 (0:00:00.089)       0:00:00.118 ******* 

TASK [Check that jinja is not too old (inst

In [16]:
# This is for resetting the kubespray cluster. 
# Use this if you are troubleshooting your Kubernetes cluster
# and you want to redeploy fresh.

# nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/reset_kubespray.sh", remote_file_path="reset.sh");
# nodes_dict['control']['node'].execute(f"chmod +x reset.sh && ./reset.sh");

In [17]:
# Add the relevant etcd data to the dynamos node
# nodes_dict['dynamos']['node'].upload_file(local_file_path="node_scripts/define_etcd_data.sh", remote_file_path="define_etcd_data.sh");
# nodes_dict['dynamos']['node'].execute(f"chmod +x define_etcd_data.sh && ./define_etcd_data.sh");

In [15]:
# Add the relevant etcd data to the dynamos node

# help(nodes_dict['dynamos']['node'])
# upload etcd files from filesystem instead of reading them from github
nodes_dict['dynamos']['node'].upload_directory(local_directory_path="../configuration/etcd_launch_files", remote_directory_path="./")
nodes_dict['dynamos']['node'].execute("ls etcd_launch_files")

nodes_dict['dynamos']['node'].upload_file(local_file_path="node_scripts/define_etcd_data_local.sh", remote_file_path="define_etcd_data_local.sh");
nodes_dict['dynamos']['node'].execute(f"chmod +x define_etcd_data_local.sh && ./define_etcd_data_local.sh");

agreements.json
archetype.json
datasets.json
microservices.json
optional_microservices.json
requestType.json
Create etcd-data folder
Copy config files into the folder
changing mode
Done!


In [39]:
%%time
# Preconfigure Helm for DYNAMOS and clone DYNAMOS repo
nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/install_dynamos.sh", remote_file_path="dynamos.sh");
nodes_dict['control']['node'].execute(f"chmod +x dynamos.sh && ./dynamos.sh")


Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Hit:1 https://prod-cdn.packages.k8s.io/repositories/isv:/kubernetes:/core:/stable:/v1.30/deb  InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (8.5.0-2ubuntu10.6).
gpg is already the newest version (2.4.4-2ubuntu17.3).
apt-transport-https is already the newest version (2.8.3).
The following packages were automatically installed and are no longer required:
  bridge-utils dns-root-data dnsmasq-base pigz ubuntu-fan
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
deb [signed-by=/usr/share/keyrings/helm.gpg] https://packages.buildkite.com/helm-linux/helm-

("Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease\nHit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease\nHit:4 http://security.ubuntu.com/ubuntu noble-security InRelease\nHit:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease\nHit:1 https://prod-cdn.packages.k8s.io/repositories/isv:/kubernetes:/core:/stable:/v1.30/deb  InRelease\nReading package lists...\nReading package lists...\nBuilding dependency tree...\nReading state information...\ncurl is already the newest version (8.5.0-2ubuntu10.6).\ngpg is already the newest version (2.4.4-2ubuntu17.3).\napt-transport-https is already the newest version (2.8.3).\nThe following packages were automatically installed and are no longer required:\n  bridge-utils dns-root-data dnsmasq-base pigz ubuntu-fan\nUse 'sudo apt autoremove' to remove them.\n0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.\ndeb [signed-by=/usr/share/keyrings/helm.gpg] https://packages.buildkite.co

In [46]:

# Configure DYNAMOS for the FABRIC nodes
agents_string = ",".join(agents)
thirdparties_string = ",".join(thirdparties)

nodes_dict['control']['node'].upload_file(local_file_path="node_scripts/configure_dynamos.sh", remote_file_path="configure_dynamos.sh"); # why upload can use the existing one
nodes_dict['control']['node'].execute(f"chmod +x configure_dynamos.sh && ./configure_dynamos.sh {agents_string} {thirdparties_string}");

Adding agents...
- agent 'server'
- agent 'clientone'
- agent 'clienttwo'
- agent 'clientthree'

Adding third parties...


In [51]:
# Optionally override the installation scripts 
nodes_dict['control']['node'].upload_file(local_file_path="../configuration/dynamos-configuration.sh", remote_file_path="/home/ubuntu/DYNAMOS/configuration/dynamos-configuration.sh")
nodes_dict['control']['node'].upload_file(local_file_path="../configuration/fill-rabbit-pvc.sh", remote_file_path="/home/ubuntu/DYNAMOS/configuration/fill-rabbit-pvc.sh");


In [52]:
%%time
# install DYNAMOS
nodes_dict['control']['node'].execute(f"~/DYNAMOS/configuration/dynamos-configuration.sh")

DYNAMOS configuration v0.1.2
Setting up paths...
definitions_example.json copied over definitions.json to ensure a clean file
Generating RabbitMQ password...
Replacing tokens...
Installing namespaces...
Release "namespaces" does not exist. Installing it now.
NAME: namespaces
LAST DEPLOYED: Tue Oct  7 16:01:21 2025
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
Preparing PVC
fill-pvc v0.1.2
pod/temp-pod created
pod/temp-pod-orch created
Waiting for temp-pod to be Running...
pod/temp-pod condition met
pod/temp-pod-orch condition met
pod "temp-pod" deleted
pod "temp-pod-orch" deleted
Installing Prometheus...
"prometheus-community" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "prometheus-community" chart repository
Update Complete. ⎈Happy Helming!⎈
Release "prometheus" does not exist. Installing it now.
NAME: prometheus
LAST DEPLOYED: Tue Oct  7 16:02:04 2025

('DYNAMOS configuration v0.1.2\nSetting up paths...\ndefinitions_example.json copied over definitions.json to ensure a clean file\nGenerating RabbitMQ password...\nReplacing tokens...\nInstalling namespaces...\nRelease "namespaces" does not exist. Installing it now.\nNAME: namespaces\nLAST DEPLOYED: Tue Oct  7 16:01:21 2025\nNAMESPACE: default\nSTATUS: deployed\nREVISION: 1\nTEST SUITE: None\nPreparing PVC\nfill-pvc v0.1.2\npod/temp-pod created\npod/temp-pod-orch created\nWaiting for temp-pod to be Running...\npod/temp-pod condition met\npod/temp-pod-orch condition met\npod "temp-pod" deleted\npod "temp-pod-orch" deleted\nInstalling Prometheus...\n"prometheus-community" already exists with the same configuration, skipping\nHang tight while we grab the latest from your chart repositories...\n...Successfully got an update from the "prometheus-community" chart repository\nUpdate Complete. ⎈Happy Helming!⎈\nRelease "prometheus" does not exist. Installing it now.\nNAME: prometheus\nLAST DEP

In [44]:
# Optional to clean up: uninstall DYNAMOS

command = "helm uninstall agents api-gateway core orchestrator namespaces prometheus thirdparties"
# nodes_dict['control']['node'].execute(command)

release "agents" uninstalled
release "api-gateway" uninstalled
release "core" uninstalled
release "orchestrator" uninstalled
These resources were kept due to the resource policy:
[Namespace] core
[Namespace] orchestrator
[Namespace] clienttwo
[Namespace] clientthree
[Namespace] uva
[Namespace] vu
[Namespace] surf
[Namespace] ingress
[Namespace] api-gateway
[Namespace] alpha
[Namespace] server
[Namespace] clientone

release "namespaces" uninstalled
release "prometheus" uninstalled
release "thirdparties" uninstalled


('release "agents" uninstalled\nrelease "api-gateway" uninstalled\nrelease "core" uninstalled\nrelease "orchestrator" uninstalled\nThese resources were kept due to the resource policy:\n[Namespace] core\n[Namespace] orchestrator\n[Namespace] clienttwo\n[Namespace] clientthree\n[Namespace] uva\n[Namespace] vu\n[Namespace] surf\n[Namespace] ingress\n[Namespace] api-gateway\n[Namespace] alpha\n[Namespace] server\n[Namespace] clientone\n\nrelease "namespaces" uninstalled\nrelease "prometheus" uninstalled\nrelease "thirdparties" uninstalled\n',
 '')

In [ ]:
# Optional delete etcd PVCs 
# nodes_dict['control']['node'].execute("kubectl get pvc --all-namespaces")

# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-0 -n core")
# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-1 -n core")
# nodes_dict['control']['node'].execute("kubectl delete pvc etcd-data-etcd-2 -n core")


# nodes_dict['control']['node'].execute("kubectl get pvc --all-namespaces")